In [0]:
df = spark.read.option("multiline", "true").json("dbfs:/FileStore/FileStore/movies-1.json")

In [0]:
df.printSchema()

root
 |-- cast: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- extract: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- href: string (nullable = true)
 |-- thumbnail: string (nullable = true)
 |-- thumbnail_height: long (nullable = true)
 |-- thumbnail_width: long (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [0]:
df.show(10)

+-------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+----+
|         cast|             extract|              genres|                href|           thumbnail|thumbnail_height|thumbnail_width|               title|year|
+-------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+----+
|           []|                NULL|                  []|                NULL|                NULL|            NULL|           NULL|After Dark in Cen...|1900|
|           []|                NULL|                  []|                NULL|                NULL|            NULL|           NULL|Boarding School G...|1900|
|           []|                NULL|                  []|                NULL|                NULL|            NULL|           NULL|Buffalo Bill's Wi...|1900|
|           []|                NULL|          

In [0]:
df.na.drop()

DataFrame[cast: array<string>, extract: string, genres: array<string>, href: string, thumbnail: string, thumbnail_height: bigint, thumbnail_width: bigint, title: string, year: bigint]

In [0]:
df = df.dropDuplicates(["title", "year"])

In [0]:
from pyspark.sql.functions import when;
df = df.withColumn("centure", when(df["year"] >= 2000, "21").otherwise("20"))
df.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+----+-------+
|                cast|             extract|              genres|                href|           thumbnail|thumbnail_height|thumbnail_width|               title|year|centure|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+----+-------+
|                  []|                NULL|                  []|                NULL|                NULL|            NULL|           NULL| The Automobile Race|1904|     20|
|                  []|Mrs. Jones Entert...|[Comedy, Short, S...|Mrs._Jones_Entert...|https://upload.wi...|             189|            320|Mrs. Jones Entert...|1909|     20|
|                  []|                NULL|                  []|                NULL|                NULL|            NULL|       

In [0]:
df_filtered = df.filter((df.year >= 2015))
df_filtered.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+----+-------+
|                cast|             extract|              genres|                href|           thumbnail|thumbnail_height|thumbnail_width|               title|year|centure|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+----+-------+
|[Jesse Eisenberg,...|American Ultra is...|    [Action, Comedy]|      American_Ultra|https://upload.wi...|             380|            256|      American Ultra|2015|     21|
|[Ryan Reynolds, J...|Pokémon Detective...|[Comedy, Fantasy,...|Detective_Pikachu...|https://upload.wi...|             385|            260|Pokémon Detective...|2019|     21|
|[Amy Ryan, Nick R...|Strange but True ...|          [Thriller]|Strange_but_True_...|https://upload.wi...|             383|       

In [0]:
writeConfig = {
    "Endpoint": "https://it-step-cosmos-db2.documents.azure.com:443/",
    "Masterkey": "",
    "Database": "MediaDB",
    "Collection": "Movies",
    "Upsert": "true"
}

In [0]:
df.write.format("com.microsoft.azure.cosmosdb.spark").options(**writeConfig).save()

In [0]:
readConfig = {
    "Endpoint": "https://it-step-cosmos-db2.documents.azure.com:443/",
    "Masterkey": "",
    "Database": "MediaDB",
    "Collection": "Movies"
}

In [0]:
df_cosmos = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**readConfig).load()
